In [1]:
import keras
import random
import numpy as np
import cv2
import os

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from imutils import paths
from keras.utils import to_categorical

#如果没有这两行代码，可能会报错OSError: image file is truncated
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
norm_size = 32
CLASS_NUM = 3
EPOCHS = 35
INIT_LR = 1e-3
BS = 32

In [3]:
#图像模型部分
from keras.layers import *

model = ResNet50()
#加一个dense层吧
img_in = Input(shape=(None,None,None,))
x = model(inputs = [img_in])
img_output = Dense(10)(x)


102858752/102853048 [==============================] - 87s 1us/step


In [4]:
#用BERT来做文本处理
from keras_bert import load_trained_model_from_checkpoint, Tokenizer

config_path = '/openbayes/input/input1/BERT/uncased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/openbayes/input/input1/BERT/uncased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/openbayes/input/input1/BERT/uncased_L-12_H-768_A-12/vocab.txt'

bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

bert_x1_in = Input(shape=(None,))
bert_x2_in = Input(shape=(None,))

x = bert_model([bert_x1_in, bert_x2_in])
text_output = Lambda(lambda x: x[:, 0])(x) # 取出[CLS]对应的向量用来做分类

In [5]:
from keras.models import Model

#将文字处理部分与图像处理部分连接起来
x = keras.layers.concatenate([img_output, text_output])

#想加一个flatten
#x = Flatten()(x)

#顶层套一个全连接层
p = Dense(3, activation='softmax')(x)

#重新定义一个模型，BERT本身有两个输入
model = Model([bert_x1_in, bert_x2_in, img_in], p)

In [7]:
from keras.optimizers import Adam

model.compile(
    optimizer=Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
             )